#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

## Задачи для совместного разбора

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [ ]:
import torch

In [ ]:
class Neuron(object):
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def forward(self, inputs):
    return inputs @ self.weights + self.bias

In [ ]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

In [ ]:
neuron = Neuron(weights, bias)
neuron.forward(inputs)

tensor(4.8400)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ReLU:
  def forward(self, inputs):
    return torch.clip(inputs, min=0)

In [ ]:
t = torch.normal(0, 1, size=(4, 3))
t

tensor([[ 0.9731, -0.7005, -1.5857],
        [ 0.9259,  0.1155, -0.0717],
        [-1.2611,  0.9651,  0.0116],
        [ 0.0670, -0.5022, -0.6518]])

In [ ]:
ReLU().forward(t)

tensor([[0.9731, 0.0000, 0.0000],
        [0.9259, 0.1155, 0.0000],
        [0.0000, 0.9651, 0.0116],
        [0.0670, 0.0000, 0.0000]])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [ ]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return ((y_pred - y_true) ** 2).sum() / len(y_pred)

In [ ]:
y_pred = torch.tensor([1.0, 3.0, 5.0])
y_true = torch.tensor([2.0, 3.0, 4.0])

In [ ]:
MSELoss().forward(y_pred, y_true)

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1_1"></p>

### 1.1
Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [1]:
import torch

class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):

        seed = 42
        torch.manual_seed(seed)

        self.n_neurons = n_neurons
        self.n_features = n_features
        self.weights = torch.randn(n_neurons, n_features)
        self.use_bias = bias

        if self.use_bias:
            self.bias = torch.rand(n_neurons).reshape(-1, 1)
        else:
            self.bias = None

    def forward(self, inputs):
        outputs = torch.matmul(inputs, self.weights.t())

        if self.use_bias:
            outputs += self.bias

        return outputs

In [2]:
# inputs: batch_size(3) x 4
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [3]:
neuron = Linear(7, 4)
neuron.forward(inputs)

tensor([[ 2.3398, -5.9315, -2.1411, -8.4206,  0.4962, -2.3766, -3.7085],
        [ 6.1785, -7.9822,  4.3246,  0.1192, -3.4674, -0.0090,  1.5405],
        [ 5.7821, -3.2093,  5.4075, -5.5589, -2.3863, -2.2812,  2.6022]])

In [ ]:
neuron = Linear(3, 4, True)
neuron.forward(inputs)

tensor([[ 1.9788,  3.6394,  4.7298],
        [ 1.8131, -6.3921,  3.6134],
        [ 0.7910,  9.3375,  1.5059]])

<p class="task" id="1_2"></p>

### 1.2
Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

In [ ]:
l1 = Linear(5, 4)
o1 = l1.forward(inputs)
o1

tensor([[ 2.3398,  8.0025, -1.8937,  1.2468, -0.2449],
        [ 6.1785,  6.4358, -1.7451,  1.7168, -4.6446],
        [ 5.7821,  5.3468, -4.2089, -1.0579,  2.5447]])

In [ ]:
l2 = Linear(7, 5)
o2 = l2.forward(o1)
o2

tensor([[ 11.9135,  -1.5360, -11.2814,  14.1880,  -4.2043,  12.3739,  -3.0027],
        [ 13.1396, -14.0535,  -7.5775,   6.7284,  -2.7874,   1.1133,  -6.9868],
        [ 19.2565,   4.3766,  -8.0752,  18.7578,  -0.8045,  19.5503,  -4.6003]])

### Создание функций активации

<p class="task" id="2_1"></p>

### 2.1
Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

In [ ]:
class Softmax:
    def forward(self, inputs):
        inputs_e = torch.exp(inputs)
        return inputs_e / torch.sum(inputs_e, dim=1).reshape(-1, 1)

In [ ]:
inputs_2 = torch.randn(4, 3)
inputs_2

tensor([[-0.9890,  0.9580,  1.3221],
        [ 0.8172, -0.7658, -0.7506],
        [ 1.3525,  0.6863, -0.3278],
        [ 0.7950,  0.2815,  0.0562]])

In [ ]:
l = Softmax()
l.forward(inputs_2)

tensor([[0.0553, 0.3873, 0.5574],
        [0.7073, 0.1452, 0.1475],
        [0.5882, 0.3022, 0.1096],
        [0.4817, 0.2882, 0.2301]])

<p class="task" id="2_1"></p>

### 2.2
Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ELU:
    def __init__(self, alpha: float):
        self.alpha = alpha

    def forward(self, inputs):
        elu_outputs = torch.where(inputs > 0, inputs, self.alpha * (torch.exp(inputs) - 1))
        return elu_outputs

In [ ]:
inputs_3 = torch.randn(4, 3)
inputs_3

tensor([[ 0.5227, -0.2384, -0.0499],
        [ 0.5263, -0.0085,  0.7291],
        [ 0.1331,  0.8640, -1.0157],
        [-0.8887,  0.1498, -0.2089]])

In [ ]:
l = ELU(alpha = 0.1)
l.forward(inputs_3)

tensor([[ 5.2272e-01, -2.1208e-02, -4.8679e-03],
        [ 5.2634e-01, -8.4628e-04,  7.2906e-01],
        [ 1.3314e-01,  8.6398e-01, -6.3784e-02],
        [-5.8883e-02,  1.4978e-01, -1.8852e-02]])

### Создание функции потерь

<p class="task" id="3_1"></p>

### 3.1
Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$
<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

In [ ]:
class CrossEntropyLoss:
  def forward(self, y_pred, y_true):
    L_i = -(torch.sum(y_true * torch.log(y_pred), dim=1))
    L = torch.sum(L_i) / L_i.size(0)
    return L

In [ ]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                        [2, 5, -1, 2],
                        [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([1, 0, 0])

l1 = Linear(3, 4)
o1 = l1.forward(inputs)

l2 = Softmax()
o2 = l2.forward(o1)

l3 = CrossEntropyLoss()
l3.forward(o2, y)

tensor(4.5292)

<p class="task" id="3_2"></p>

### 3.2
Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

In [ ]:
class MSERegularized:
    def __init__(self, lambda_):
        self.lambda_ = lambda_

    def data_loss(self, y_pred, y_true):
        return torch.mean((y_true - y_pred)**2)

    def reg_loss(self, weights):
        return self.lambda_ * torch.sum(weights**2)

    def forward(self, y_pred, y_true, weights):
        return self.data_loss(y_pred, y_true) + self.reg_loss(weights)

In [ ]:
y_pred = torch.tensor([-0.5, 1, 1.7])
y_true = torch.tensor([0, 0.6, 2.3])
weights = torch.normal(0, 5, (10, 1))

l4 = MSERegularized(0.5)
l4.forward(y_pred, y_true, weights)

tensor(148.3832)

## Обратная связь
- [ ] Хочу получить обратную связь по решению